In [1]:
import sys

sys.path.append("../src/lake_modelling/utils")

import altair as alt
import lake_model as lm
import lmfit
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sn
from joblib import Parallel, delayed
from scipy import integrate

plt.style.use("ggplot")

# Omregningsfaktorer

Plots of "omregningsfaktorer" are useful for comparing the effectiveness of different lime products in lakes with different properties. The plots assume an initial lake pH, and a pH target that is to be achieved after some period of time (e.g. one year). For a range of lakes with different characteristics, the model optimises the amount of each lime product needed to achieve the target. These estimates are then compared to simulations for a reference "standard" product and the omregningsfaktor (OF) calculated as:

$$OF = \frac{Mass_{test-prod}}{Mass_{ref-prod}}$$

OF values less than one imply that the test product is more effective than the standard (per unit mass), whereas value greater than one suggest it is less effective.

The plots in this notebook make the following assumptions:

 * The initial lake pH and inflow pH are both 4.5 and the TOC concentration is 4 mg/l (i.e. the middle TOC class).
 * The liming target is a **pH of 6 after one year**.
 * The reference product is always `Standard Kalk Kat3`.
 * Lakes are assumed to have a `Fjell` flow profile and liming occurs in July with `Wet` spreading over the entire surface area.
 * Results are generated for a range of **depths** (5, 10, 15 and 20 metres) and **water residence times** (0.3, 0.5, 0.7, 1, 1.5 and 2 years).

In [2]:
# Define functions to optimise model


def run_model(
    lime_dose,
    product,
    area,
    depth,
    tau,
    flow_prof,
    meth,
    spr_prop,
    rate_const,
    act_const,
):
    prod = lm.LimeProduct(product)
    lake = lm.Lake(
        area=area,
        depth=depth,
        tau=tau,
        flow_prof=flow_prof,
        pH_lake0=5.8,
        pH_inflow=5,
        toc_lake0=4,
    )
    model = lm.Model(
        lake,
        prod,
        lime_dose=lime_dose,
        lime_month=7,
        spr_meth=meth,
        spr_prop=spr_prop,
        F_sol=1,
        rate_const=rate_const,
        activity_const=act_const,
        ca_aq_sat=8.5,
        n_months=12,
    )
    df = model.run()

    return (model, df)


def residual(
    params,
    product,
    area,
    depth,
    tau,
    flow_prof,
    meth,
    spr_prop,
    rate_const,
    act_const,
):
    lime_dose = params["lime_dose"].value
    model, df = run_model(
        lime_dose,
        product,
        area,
        depth,
        tau,
        flow_prof,
        meth,
        spr_prop,
        rate_const,
        act_const,
    )
    final_ph = df["pH"].iloc[-1]
    residual = final_ph - 6.0

    return residual

In [3]:
init_dose = 10
area = 0.2
flow_prof = "fjell"
meth = "wet"
spr_prop = 1
rate_const = 0.1
act_const = 0.1
ref_prod = "Standard Kalk Kat3"
test_prods = [
    "Miljøkalk VK3",
    "Miljøkalk EY3",
    "Microdol1",
    "Microdol5",
    "Visnes Filterkalk Kat3",
    "Visnes Filterkalk Kat4",
    "Omya Hustadmarmor Biokalk",
]
depths = [5, 10, 15, 20]
taus = [0.3, 0.5, 0.7, 1, 1.5, 2]

In [4]:
prod_names = list(set([ref_prod] + test_prods))
res_dict = {
    "Produkt": [],
    "Dybde (m)": [],
    "Oppholdstid (år)": [],
    "optimal_dose_mgpl": [],
    "lime_tonn": [],
}
for prod_name in prod_names:
    print(prod_name)
    for depth in depths:
        for tau in taus:
            params = lmfit.Parameters()
            params.add("lime_dose", value=init_dose, min=0, max=85)
            # if prod_name.startswith("Microdol"):
            #     act_const = 0.1  # 0.6
            # else:
            #     act_const = 0.1
            result = lmfit.minimize(
                residual,
                params,
                args=(
                    prod_name,
                    area,
                    depth,
                    tau,
                    flow_prof,
                    meth,
                    spr_prop,
                    rate_const,
                    act_const,
                ),
            )
            optimal_dose = result.params["lime_dose"].value
            model, df = run_model(
                result.params["lime_dose"].value,
                prod_name,
                area,
                depth,
                tau,
                flow_prof,
                meth,
                spr_prop,
                rate_const,
                act_const,
            )
            final_ph = df["pH"].iloc[-1]
            if final_ph < 5.8:
                print("    ", tau, final_ph)
            lime_tonnes = spr_prop * model.lake.volume * optimal_dose / 1e9

            res_dict["Produkt"].append(prod_name)
            res_dict["Dybde (m)"].append(depth)
            res_dict["Oppholdstid (år)"].append(tau)
            res_dict["optimal_dose_mgpl"].append(optimal_dose)
            res_dict["lime_tonn"].append(lime_tonnes)

df = pd.DataFrame(res_dict)
df.head()

Visnes Filterkalk Kat4
Standard Kalk Kat3
Omya Hustadmarmor Biokalk
Miljøkalk EY3
Microdol5
Microdol1
Visnes Filterkalk Kat3
Miljøkalk VK3


,Produkt,Dybde (m),Oppholdstid (år),optimal_dose_mgpl,lime_tonn
0,Visnes Filterkalk Kat4,5,0.3,20.297983,20.297983
1,Visnes Filterkalk Kat4,5,0.5,7.678562,7.678562
2,Visnes Filterkalk Kat4,5,0.7,4.519877,4.519877
3,Visnes Filterkalk Kat4,5,1.0,2.836050,2.836050
4,Visnes Filterkalk Kat4,5,1.5,1.837108,1.837108


In [5]:
om_df = (
    df[["Produkt", "Dybde (m)", "Oppholdstid (år)", "lime_tonn"]]
    .set_index(["Dybde (m)", "Oppholdstid (år)", "Produkt"])
    .unstack("Produkt")
)
om_df.columns = om_df.columns.get_level_values(1)
for col in om_df.columns:
    om_df[f"{col}_fac"] = om_df[col] / om_df[ref_prod]
cols = [col for col in om_df.columns if col.endswith("_fac")]
om_df = om_df[cols]
cols = [col.replace("_fac", "") for col in om_df.columns]
om_df.columns = cols
del om_df[ref_prod]
om_df.reset_index(inplace=True)
om_df = om_df.melt(
    id_vars=["Dybde (m)", "Oppholdstid (år)"],
    var_name="Produkt",
    value_name="Faktor (-)",
)

csv_path = r"../data/omregningsfaktorer.csv"
om_df.to_csv(csv_path, index=False)

In [6]:
# Read saved data for speed
csv_path = r"../data/omregningsfaktorer.csv"
om_df = pd.read_csv(csv_path)
om_df.head()

,Dybde (m),Oppholdstid (år),Produkt,Faktor (-)
0,5,0.3,Microdol1,0.839288
1,5,0.5,Microdol1,0.931731
2,5,0.7,Microdol1,0.984707
3,5,1.0,Microdol1,1.024785
4,5,1.5,Microdol1,1.055714


In [7]:
# Plot
om_df["Dybde (m)"] = om_df["Dybde (m)"].astype(str) + " m"
checkbox_selection = alt.selection_point(fields=["Produkt"], bind="legend")
chart = (
    alt.Chart(om_df)
    .mark_line(point=True)
    .encode(
        x=alt.X(
            "Oppholdstid (år):Q", scale=alt.Scale(type="log", base=10, domain=(0.2, 3))
        ),
        y=alt.Y(
            "Faktor (-):Q",
            scale=alt.Scale(domain=(0.8, 2)),
            axis=alt.Axis(title="Faktor (-)"),
        ),
        color="Produkt:N",
        opacity=alt.condition(checkbox_selection, alt.value(1), alt.value(0)),
        tooltip=["Oppholdstid (år)", "Faktor (-)", "Produkt"],
    )
    .add_params(checkbox_selection)
    .properties(width=400, height=400)
    .facet(
        facet=alt.Facet("Dybde (m):N", sort=[5, 10, 15, 20]),
        columns=2,
    )
    .resolve_scale(x="independent", y="independent")
    .interactive()
)

chart

alt.FacetChart(...)